In [152]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import requests
from bs4 import BeautifulSoup
import time 
from tqdm import tqdm
import random
import pandas as pd
import re

In [2]:
Pro_URL= pd.read_excel('Links.xlsx', sheet_name='url', index_col=False)
url_list = Pro_URL['loc'].tolist()

In [7]:
Pro_URL = url_list

In [287]:
Pro_URL.index('https://www.whiteflowerfarm.com/59092-product.html')
Pro_URL[1264]
# Pro_URL.remove('https://www.whiteflowerfarm.com/copper-tray-with-handles')

'https://www.whiteflowerfarm.com/three-months-of-winter-bulb-gardens-january-march'

In [328]:
def get_product_info(URL):
    def createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount):
        row = {
            'ProductURL': URL,
            'ProductName':ProductName,
            'SKU':SKU,
            'Availability': Stock,
            'Quantity':Quantity,
            'SalePrice': SalePrice,
            'ReviewCount': ReviewCount,
            'Rating': Rating
            }
        return row
    driver.get(URL)
    time.sleep(2)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    ProductName = soup.find('h1').text
    SKU = soup.find('span', class_='item-sku js-item-sku').text[5:]
    Rating = soup.find('div', class_='ratings-box').find('a').get('data-rating')
    ReviewCount = soup.find('div', class_='ratings-box').find('a').text.strip('()\n')
    Stock = soup.find('div', class_='add-to-cart-wrap').text.strip('\n')
    if Stock == 'Add to cart':
        Stock = 'InStock'
    df = []
    skipped = []
    if len(soup.find_all('div', class_="cart-actions")[1].find_all('option')):
        if soup.find_all('div', class_="cart-actions")[1].find_all('optgroup'):
            for i in range(len(soup.find_all('div', class_="cart-actions")[1].find_all('optgroup'))):
                Quantity = soup.find_all('div', class_="cart-actions")[1].find_all('optgroup')[i].get('label')
                Qnumber = int(re.search(r'(\d+)', Quantity).group())
                price_per_unit = float(Quantity.split('$')[1].split()[0])
                SalePrice = Qnumber * price_per_unit
                SalePrice = f'${SalePrice:.2f}'
                row = createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount)
                df.append(row)
        else:
            Quantity = soup.find_all('div', class_="cart-actions")[1].find_all('option')[0].text.strip('\n').split('for')[0].strip()
            SalePrice = soup.find_all('div', class_="cart-actions")[1].find_all('option')[0].text.strip('\n').split('for')[1].strip(' total')
            row = createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount)
            df.append(row)
        return df, soup, skipped
    elif soup.find('div', class_="pc-attribute-wrapper"):
        if 'months' in ProductName.lower():
            skipped.append(URL)
#                 for i in range(len(soup.find('div', class_="pc-attribute-wrapper").find_all('option', class_=''))):
#                     Varient = soup.find('div', class_="pc-attribute-wrapper").find_all('option', class_='')[i].get('value')
#                     combo_box = driver.find_element(By.XPATH, "//select[@id='pc-attr-1362857527-0']")
#                     combo_box.send_keys(Varient)
#                     for j in range(len(soup.find_all('input', class_='pc-attribute pc-attr726057125'))):
#                         page_source = driver.page_source
#                         soup = BeautifulSoup(page_source, 'html.parser')
#                         ProductName = soup.find('h1').text
#                         SKU = soup.find('span', class_='item-sku js-item-sku').text[5:]
#                         SalePrice = soup.find('span', class_='price').text
#                         Quantity= None
#                         row = createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount)
#                         df.append(row)


            
        elif soup.find('span', class_='child-name'):
            for i in range(len(soup.find_all('span', class_='child-name'))):
                ProductName = soup.find_all('span', class_='child-name')[i].text
                SalePrice = soup.find_all('span', class_='child-price')[i].find('span').text.strip('\n')
                Quantity= None
                row = createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount)
                df.append(row)
            return df, soup, skipped
        
        else:
            for i in range(len(soup.find('div', class_="pc-attribute-wrapper").find_all('option', class_=''))):
                Varient = soup.find('div', class_="pc-attribute-wrapper").find_all('option', class_='')[i].get('value')
                combo_box = driver.find_element(By.XPATH, '/html/body/section[1]/div/div/form/div[1]/div[2]/div[5]/div[2]/div/div/div/select')
                combo_box.send_keys(Varient)
                time.sleep(2)
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
                SKU = soup.find('span', class_='item-sku js-item-sku').text[5:]
                SalePrice = soup.find('span', class_='price').text
                Quantity= None
                row = createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount)
                df.append(row)
        return df, soup, skipped
    else:
        if soup.find('span', class_='price',id="store_price"):
            SalePrice = soup.find('span', class_='price',id="store_price").text
            Quantity= None
            row = createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount)
            df.append(row)
            return df, soup, skipped
        elif soup.find('span', class_="price sale-price"):
            SalePrice = soup.find('span', class_="price sale-price").text
            Quantity= None
            row = createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount)
            df.append(row)
            return df, soup, skipped
        else:
            SalePrice = None
            Quantity= None
            row = createRow(ProductName,SKU,Stock,Quantity, SalePrice,Rating, ReviewCount)
            df.append(row)
            return df, soup, skipped

In [205]:
Pro_URL[1037]

'https://www.whiteflowerfarm.com/months-of-easy-care-moth-orchids'

In [352]:
Product_details4=[]

In [262]:
skipped = []

In [362]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[3465:], desc="GettingInfo", unit="URL"):
    df, soup,skipped = get_product_info(i)
    Product_details4.extend(df)
    skipped.extend(skipped)
    print(df)
driver.quit()
daf = pd.DataFrame(Product_details4)
daf.to_csv('WhiteFlowerFarm4.csv',index=False)

GettingInfo:   1%|▌                                                                   | 1/130 [00:07<15:18,  7.12s/URL]

[{'ProductURL': 'https://www.whiteflowerfarm.com/all-season-birdseed-casita', 'ProductName': 'New! All-Season Birdseed Casita', 'SKU': 'S49039', 'Availability': 'InStock', 'Quantity': None, 'SalePrice': '$42.00', 'ReviewCount': '0 Reviews', 'Rating': '0'}]


GettingInfo:   2%|█                                                                   | 2/130 [00:11<11:58,  5.61s/URL]

[{'ProductURL': 'https://www.whiteflowerfarm.com/flower-herb-drying-rack', 'ProductName': 'New! Flower & Herb Drying Rack', 'SKU': 'S49041', 'Availability': 'InStock', 'Quantity': None, 'SalePrice': '$22.00', 'ReviewCount': '0 Reviews', 'Rating': '0'}]


GettingInfo:   2%|█▌                                                                  | 3/130 [00:16<10:46,  5.09s/URL]

...

[{'ProductURL': 'https://www.whiteflowerfarm.com/vegtrug-raised-bed', 'ProductName': 'VegTrug Raised Bed, natural', 'SKU': 'S51202', 'Availability': 'InStock', 'Quantity': None, 'SalePrice': '$265.00', 'ReviewCount': '0 Reviews', 'Rating': '0'}, {'ProductURL': 'https://www.whiteflowerfarm.com/vegtrug-raised-bed', 'ProductName': 'VegTrug Raised Bed, gray', 'SKU': 'S51202', 'Availability': 'InStock', 'Quantity': None, 'SalePrice': '$265.00', 'ReviewCount': '0 Reviews', 'Rating': '0'}]


GettingInfo:  99%|█████████████████████████████████████████████████████████████████▍| 129/130 [11:39<00:07,  7.56s/URL]

[{'ProductURL': 'https://www.whiteflowerfarm.com/dinnerplate-dahlia-collection-2', 'ProductName': 'Dinnerplate Dahlia Collection 2.0, 4 tubers', 'SKU': 'S83323', 'Availability': 'InStock', 'Quantity': None, 'SalePrice': '$45.00', 'ReviewCount': '0 Reviews', 'Rating': '0'}]


GettingInfo: 100%|██████████████████████████████████████████████████████████████████| 130/130 [11:44<00:00,  5.42s/URL]

[{'ProductURL': 'https://www.whiteflowerfarm.com/german-primrose-libre-red-in-basket', 'ProductName': 'German Primrose Libre™ Red in basket ', 'SKU': 'S83345', 'Availability': 'InStock', 'Quantity': None, 'SalePrice': '$46.00', 'ReviewCount': '0 Reviews', 'Rating': '0'}]


In [347]:
Pro_URL[2868]

'https://www.whiteflowerfarm.com/red-bird-copper-wall-art'

In [361]:
driver.quit()

In [363]:
import os
import pandas as pd

def combine_csv_files(input_folder, output_file):
    # Get a list of all CSV files in the input folder starting with "FastGrowingTrees"
    csv_files = [file for file in os.listdir(input_folder) if file.startswith('WhiteFlowerFarm') and file.endswith('.csv')]

    # Initialize an empty list to store DataFrames
    data_frames = []

    # Loop through each matching CSV file and append its data to the data_frames list
    for file in csv_files:
        file_path = os.path.join(input_folder, file)
        current_data = pd.read_csv(file_path)
        print(file ,len(current_data))
        data_frames.append(current_data)

    # Concatenate all DataFrames in the list along the rows
    combined_data = pd.concat(data_frames, ignore_index=True)

    # Save the combined data to a new CSV file
    combined_data.to_csv(output_file, index=False)
    print(f"Combined data saved to {output_file}")

# Example usage
input_folder = r'C:\Users\Rahul Manocha\Documents\KDMS\WebScrapping\1st time\WhiteFlowerFarm'
output_file = 'WhiteFlowerFarm.csv'
combine_csv_files(input_folder, output_file)

WhiteFlowerFarm1.csv 1108
WhiteFlowerFarm2.csv 1311
WhiteFlowerFarm3.csv 5213
WhiteFlowerFarm4.csv 1329
Combined data saved to WhiteFlowerFarm.csv
